# Семинар 7. Временные ряды и реккурентные сети

**Внимание!** Для работы над задачами этого семинара на локальном устройстве потребуется установить [`PyTorch`](https://pytorch.org/get-started/locally/), если он ещё не был установлен вами при работе над задачами Семинаров 5 и 6.

При работе в Google Colab ничего устанавливать не нужно, модуль уже доступен и его можно сразу импортировать.

После установки `PyTorch` убедитесь, что следующий код запускается и выводит на экран набор нормально распределенных вещественных чисел размером 5×3:

```python
import torch
x = torch.rand(5, 3)
print(x)
```

В этом семинаре вам предлагается поработать над временными рядами на примере [датасета](https://srhdpeuwpubsa.blob.core.windows.net/whdh/COVID/WHO-COVID-19-global-daily-data.csv) по COVID-19 от Всемирной Организации Здравоохранения. Сначала мы познакомимся с особенностями работы с временными рядами, а затем устроим соревнование между ML и DL моделями в задаче прогнозирования.

## Задача 7.1 Базовая обработка временного ряда [max = 15 баллов]

**Внимание!** Каждый пункт в данной задаче оценивается только при успешном выполнении всех предыдущих

Загрузите данные по заболеваемости COVID-19 с сайта Всемирной Организации Здравоохранения. Можно сделать это вручную или использовать следующий код:



```shell
!wget -O data.csv https://srhdpeuwpubsa.blob.core.windows.net/whdh/COVID/WHO-COVID-19-global-daily-data.csv
```

После скачивания подгрузите данные в блокнот для дальнейшей работы. Вероятно, в этом упражнении вам будет удобно работать с библиотекой `pandas`, но это не обязательно – принимается любое корректное решение.

[ 1 балл ] Выделите из данных две колонки, отвечающие за дату и число новых зарегистрированных случаев заражения

[2 балла] Обратите внимание, что таблица содержит данные, относящиеся к разным странам. Это создает некоторую неоднородность информации и, если ничего не предпринять, будет мешать нам в дальнейшем. Просуммируйте новые случаи по странам, чтобы получился единый глобальный временной ряд (дата и общее число новых случаев)

[ 1 балл ] Теперь ограничимся наиболее интересным промежутком времени, когда пандемия была в самом разгаре: из всего временного ряда выделите только промежуток с 01 марта 2020 года по 28 февраля 2023 года включительно

[ 1 балл ] Визуализируйте получившийся временной ряд в осях дата–число зарегистрированных заболевших. Используйте линейный масштаб по обеим осям, подпишите оси. Что можно «на глаз» сказать о стационарности временного ряда?


In [ ]:
# ваш код

[ 1 балл ] Разделите данные на тренировочный, валидационный и тестовый фрагменты в соотношении ⅔ ÷ ⅙ ÷ ⅙. А именно, тренировочный фрагмент отправьте данные с 01 марта 2020 года по 28 февраля 2022 года (два года), в валидационный фрагмент – с 01 марта 2022 года по 31 августа 2022 года (полгода), в тестовый фрагмент – оставшуюся часть ряда (еще полгода)


[2 балла] При визуализации вы могли заметить, что значения во временном ряду испытывают значимые высокочастотные (относительно выбранного нами промежутка времени) колебания. С чем это может быть связано? Каков период наблюдаемых колебаний? Вычислите период высокочастотных колебаний количественно (вычисляя автокоррелляцию, применяя преобразовани Фурье, или иным способом на ваше усмотрение), пользуясь только тренировочным фрагментом

[ 1 балл ] Устраните малые колебания во всех фрагментах данных при помощи скользящего среднего по временному окну, равному найденному в предыдущем пункте периоду. Обратите внимание: тренировочный, валидационный и тестовый фрагменты должны усредняться отдельно и независимо. Никакие данные не должны перетекать из одного фрагмента в другой при усреднении!


[ 1 балл ] Визуализируйте получившийся усредненный временной ряд. Разными цветами покажите тренировочный, валидационный и тестовый фрагменты.

In [ ]:
# ваш код

[ 1 балл ] В получившемся временном ряду мы видим еще одну проблему: значения имеют разброс в несколько порядков величины. Какую монотонную функцию можно применить ко всем значениям ряда, чтобы привести все значения к одному порядку величины? Реализуйте это преобразование

[ 1 балл ] Визуализируйте получившийся преобразованный временной ряд, снова разными цветами покажите разные фрагменты данных

In [ ]:
# ваш код

[3 балла] При помощи [Augmented Dickey–Fuller test](https://en.wikipedia.org/wiki/Augmented_Dickey%E2%80%93Fuller_test) оцените стационарность получившегося ряда, пользуясь только тренировочным фрагментом. Проинтерпретируйте результаты теста с точки зрения их статистической значимости. Можно (но не обязательно) пользоваться модулем `statsmodels`.

При желании можно сделать дополнительные преобразования над временным рядом, на ваше усмотрение

In [ ]:
# ваш код

## Задача 7.2 Baseline модель [max = 15 баллов]

**Внимание!** Данная задача оценивается только при условии успешного выполнения Задачи 7.1. Каждый пункт в данной задаче оценивается только при условии успешного выполнения всех предыдущих

Для начала оценим на подготовленных данных примитивную модель – это позволит нам получить минимальную допустимую границу качества для будущих экспериментов. Пусть это будет модель без обучаемых параметров, которая принимает на вход последовательность длины 7 – (предобработанные!) данные за последнюю неделю, и выдает их арифметическое среднее в качестве прогноза на следующий, восьмой день.

[5 баллов] Дополните / отредактируйте предложенные шаблоны для подготовки датасета, который мы будем подавать на вход модели. В шаблонах предполагается, что вы работаете с типом `pandas.Series` в качестве временных рядов, но вы можете изменить это на своё усмотрение.

```python
def make_folds(series : pandas.Series, sequence_length : int = 7):

    for i in range(len(series)-sequence_length):
        input_fold = # TO-DO
        output_fold = # TO-DO
        yield input_fold, output_fold


def folds_to_tabular(folds) -> tuple[numpy.ndarray]:
    X, y = [], []
    for input_fold, output_fold in folds:
        # TO-DO
    return np.asarray(X), np.asarray(y).ravel()
```

Затем подготовьте псевдо-табличные данные с помощью ваших функций. Например, так:
```python
X_train, y_train = folds_to_tabular(make_folds(train_series))
X_val, y_val = folds_to_tabular(make_folds(val_series))
X_test, y_test = folds_to_tabular(make_folds(test_series))
```


In [ ]:
# ваш код

[5 баллов] Создайте класс baseline модели, пользуясь предложенным шаблоном. Обратите внимание, что метод `fit` здесь чисто декоративный – ведь модель не имеет обучаемых параметров! Не забудьте, тем не менее, устроить проверку входной длины последовательности.

```python
class BaselineForecast:
    def __init__(self, sequence_length : int = 7):
        self.sequence_length = # TO-DO

    def fit(self, X : numpy.ndarray, y : numpy.ndarray):
        # TO-DO
        return self

    def predict(self, X : numpy.ndarray) -> numpy.ndarray:
        pass
        # TO-DO
```

Визуализируйте предсказания модели на одном графике с реальными данными, снова выделите разными цветами тренировочный, валидационный и тестовый фрагменты (по аналогии с Задачей 7.1). Затем создайте экземпляр класса и оцените mean absolute error в исходных единицах (число заболевших) на валидационном и тестовом фрагменте. Приемлемо ли такое качество?

In [ ]:
# ваш код

[5 баллов] Протестируйте ту же самую модель в режиме экстраполяции предсказаний. То есть, используйте только первый фолд из тестового набора данных, и постепенно дополняйте его предсказаниями самой модели в качестве псевдо-данных. Это позволит модели сделать предсказание не на один день вперед, а сразу на большой горизонт, но ошибка будет накапливаться.

Оформите экстраполятор предсказаний модели в виде функции по предложенному шаблону.


```python
def extrapolate_predictions(model, X):
    assert hasattr(model, "predict")
    n_folds, _ = X.shape
    data = X[:1]
    pred = []
    for i in range(n_folds):
        pred = model.predict(data)
        data = # TO-DO
    return np.asarray(pred)
```

Получите предсказания на тестовом фрагменте в режиме экстраполяции и снова оцените mean absolute error в единицах числа заболевших. Улучшилось или ухудшилось качество по сравнению с предыдущим пунктом? Добавьте на график, построенный в предыдщем пункте, еще одну линию – предсказания на тестовом фрагменте в режиме экстраполяции.

Постройте еще один, отдельный график – зависимость абсолютной ошибки одного предсказания на тестовом фрагменте от даты в стандартном режиме и в режиме экстраполяции (две кривые в одних осях)

In [ ]:
# ваш код

## Задача 7.3 Классические ML модели [max = 15 баллов]

**Внимание!** Данная задача оценивается только при условии успешного выполнения Задач 7.1 и 7.2.

[5 баллов] Обучите на подготовленных в Задаче 7.2 данных модель `LinearRegression` из `sklearn.linear_model`. Модель должна по-прежнему делать предсказания на один день вперед, основываясь на данных за предшествующую неделю. Оцените при помощи той же самой метрики качество на валидации и тесте в стандартном режиме предсказаний, а также качество на тесте в режиме экстраполяции. Сравните с baseline моделью. Нарисуйте аналогичные Задаче 7.2 графики

In [ ]:
# ваш код

[2 балла] Заметим, что нашу baseline модель можно интерпретировать как линейную модель с зафиксированными параметрами. Чему равны параметры в такой интерпретации? Сравните их с получившимися оптимальными параметрами линейной регресии. Как можно интерпретировать результаты? Какие дни оказываются более «важны» для прогноза в baseline модели? В модели линейной регресии?  

**Внимание!** Данный пункт оценивается только при успешном выполнении предыдущего.

In [ ]:
# ваш код

[5 баллов] Обучите на подготовленных в Задаче 7.2 данных модель `RandomForestRegressor` из `sklearn.ensemble`. Модель должна по-прежнему делать предсказания на один день вперед, основываясь на данных за предшествующую неделю. Оцените при помощи той же самой метрики качество на валидации и тесте в стандартном режиме предсказаний, а также качество на тесте в режиме экстраполяции. Сравните с baseline моделью. Нарисуйте аналогичные Задаче 7.2 графики

In [ ]:
# ваш код

[3 балла] Оптимизируйте гиперпараметр `n_estimators` (на ваше усмотрение можно добавить и другие параметры) с помощью метрики mean absolute error в исходных единицах на валидационной выборке при помощи [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) и [`PredefinedSplit`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.PredefinedSplit.html), либо других автоматизированных методов. Просканируйте не менее 10 значений гиперпараметра `n_estimators`.

Добейтесь mean absolute error на тесте **не более 250 тысяч заболевших** в стандартном режиме предсказаний. Сравните с baseline моделью. Нарисуйте аналогичные Задаче 7.2 графики

In [ ]:
# ваш код

## Задача 7.4 Простая RNN-модель [max = 15 баллов]

**Внимание!** Данная задача оценивается только при условии успешного выполнения Задач 7.1 и 7.2.

Обучите простую RNN-модель на той же задаче прогнозирования (на один день вперед по данным за предшествующую неделю). Воспользуйтесь для этого модулем `torch`.

[3 балла] Дополните предложенный класс для создания датасета:


```python
class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, sequence : numpy.ndarray, sequence_length : int = 7):
        super().__init__()
        self.sequence_length = # TO-DO
        self.sequence = # TO-DO

    def __getitem__(self, idx : int) -> tuple[torch.Tensor]:
        data = # TO-DO
        labels = # TO-DO
        return data, labels

    def __len__(self) -> int:
        return len(self.sequence) - self.sequence_length
```

[2 балла] При помощи `torch.utils.data.DataLoader` создайте тренировочный, валидационный и тестовый `dataloader`. Для самопроверки можно использовать следующий код:



```python
data, labels = next(iter(train_loader))
data.shape, labels.shape
```



In [ ]:
# ваш код

[3 балла] Создайте класс модели `RNNForecast` с одним реккурентным слоем `torch.nn.RNN`. Помните, что рекурсивный слой возвращает два тезора: `output_state` и `hidden_state`. Предсказание сделайте на основе `hidden_state`, отправляя его выходы в полносвязный слой. Число нейронов в скрытом слое оставьте в качестве гиперпараметра, который можно настраивать. Проверьте, что батч тренировочных данных проходит через экземпляр вашей модели:


```python
data, labels = next(iter(train_loader))
model = # TO-DO
with torch.no_grad():
    out = model(data)
out
```

На выходе вы должны увидеть тензор размера `batch_size`


In [ ]:
# ваш код

[7 баллов] Обучите модель, визуализируйте кривые обучения на тренировочном и валидационном фрагментах. Добейтесь mean absolute error на тесте **не более 80 тысяч заболевших** в стандартном режиме предсказаний. Сравните с baseline моделью и моделями машинного обучения. Нарисуйте аналогичные Задаче 7.2 графики

In [ ]:
# ваш код

## Задача 7.5* Простая LSTM-модель [max = 15 дополнительных баллов]

**Внимание!** Данная задача оценивается только при условии успешного выполнения Задач 7.1–7.4.

Повторите задачу 7.4, используя для прогноза LSTM-блок вместо RNN-блока. Нарисуйте те же графики, оцените те же метрики, сравните с baseline-моделью, ML-моделями и RNN-моделью.

In [ ]:
# ваш код